In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# אי-שוויון CHSH

*הערכת שימוש: שתי דקות על מעבד Heron r2 (הערה: זוהי הערכה בלבד. זמן הריצה בפועל עשוי להשתנות.)*
## רקע
במדריך זה, תריץ ניסוי על מחשב קוונטי כדי להדגים את הפרת אי-שוויון CHSH עם הפרימיטיב Estimator.

אי-שוויון CHSH, הנקרא על שם המחברים Clauser, Horne, Shimony ו-Holt, משמש להוכחה ניסויית של משפט בל (1969). משפט זה קובע שתיאוריות של משתנים נסתרים מקומיים אינן יכולות להסביר חלק מהתוצאות של שזירה קוונטית במכניקת הקוונטים. הפרת אי-שוויון CHSH משמשת להוכחה שמכניקת הקוונטים אינה תואמת לתיאוריות של משתנים נסתרים מקומיים. זהו ניסוי חשוב להבנת היסודות של מכניקת הקוונטים.

פרס נובל לפיזיקה לשנת 2022 הוענק לאלן אספה, ג'ון קלאוזר ואנטון זיילינגר, בין היתר בזכות עבודתם החלוצית במדעי המידע הקוונטי, ובפרט על ניסוייהם עם פוטונים שזורים המדגימים הפרה של אי-שוויוני בל.
## דרישות מקדימות
לפני תחילת המדריך, ודא שהדברים הבאים מותקנים:

* Qiskit SDK גרסה 1.0 ומעלה, עם תמיכה ב-[visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime‏ (`pip install qiskit-ibm-runtime`) גרסה 0.22 ומעלה
## הגדרה

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## שלב 1: מיפוי קלט קלאסי לבעיה קוונטית
בניסוי זה, ניצור זוג שזור שעליו נמדוד כל קיוביט בשני בסיסים שונים. נסמן את הבסיסים של הקיוביט הראשון ב-$A$ ו-$a$ ואת הבסיסים של הקיוביט השני ב-$B$ ו-$b$. זה מאפשר לנו לחשב את הכמות CHSH $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

כל צפייה היא $+1$ או $-1$. ברור שאחד מהאיברים $B\pm b$ חייב להיות $0$, והשני חייב להיות $\pm 2$. לכן, $S_1 = \pm 2$. ערך הציפייה של $S_1$ חייב לקיים את אי-השוויון:

$$
|\langle S_1 \rangle|\leq 2.
$$

פיתוח $S_1$ במונחי $A$, $a$, $B$, ו-$b$ נותן:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

ניתן להגדיר כמות CHSH נוספת $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

מה שמוביל לאי-שוויון נוסף:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

אם מכניקת הקוונטים ניתנת לתיאור על ידי תיאוריות של משתנים נסתרים מקומיים, אי-השוויונות הקודמים חייבים להתקיים. אולם, כפי שיודגם במדריך זה, אי-שוויונות אלה ניתנים להפרה במחשב קוונטי. לכן, מכניקת הקוונטים אינה תואמת לתיאוריות של משתנים נסתרים מקומיים.
אם ברצונך ללמוד עוד תיאוריה, עיין ב-[Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) עם John Watrous.
תיצור זוג שזור בין שני קיוביטים במחשב קוונטי על ידי יצירת מצב בל $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. באמצעות הפרימיטיב Estimator, תוכל לקבל ישירות את ערכי הציפייה הנדרשים ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$, ו-$\langle ab \rangle$) לחישוב ערכי הציפייה של שתי כמויות CHSH $\langle S_1\rangle$ ו-$\langle S_2\rangle$. לפני הכנסת הפרימיטיב Estimator, היה צורך לבנות את ערכי הציפייה מתוצאות המדידה.

תמדוד את הקיוביט השני בבסיסי $Z$ ו-$X$. הקיוביט הראשון יימדד גם הוא בבסיסים אורתוגונליים, אך בזווית ביחס לקיוביט השני, אותה נסרוק בין $0$ ל-$2\pi$. כפי שתראה, הפרימיטיב Estimator מקל מאוד על הרצת מעגלים עם פרמטרים. במקום ליצור סדרה של מעגלי CHSH, עליך ליצור רק *מעגל CHSH אחד* עם פרמטר המציין את זווית המדידה וסדרת ערכי פאזה לפרמטר.

לבסוף, תנתח את התוצאות ותציג אותן מול זווית המדידה. תראה שבטווחי זוויות מדידה מסוימים, ערכי הציפייה של כמויות CHSH $|\langle S_1\rangle| > 2$ או $|\langle S_2\rangle| > 2$, מה שמדגים את הפרת אי-שוויון CHSH.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### יצירת מעגל CHSH עם פרמטר
ראשית, נכתוב את המעגל עם הפרמטר $\theta$, שנקרא לו `theta`. הפרימיטיב [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) יכול לפשט מאוד את בניית המעגל וניתוח הפלט על ידי מתן ערכי ציפייה ישירים של צפיות. בעיות רבות בעלות עניין, בייחוד עבור יישומים לטווח קרוב על מערכות רועשות, ניתנות לניסוח במונחי ערכי ציפייה. הפרימיטיב `Estimator` (V2) יכול לשנות אוטומטית את בסיס המדידה על פי הצפייה שסופקה.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### יצירת רשימת ערכי פאזה להקצאה מאוחרת
לאחר יצירת מעגל CHSH עם פרמטר, תיצור רשימת ערכי פאזה שיוקצו למעגל בשלב הבא. ניתן להשתמש בקוד הבא ליצירת רשימה של 21 ערכי פאזה בטווח $0$ עד $2 \pi$ עם ריווח שווה, כלומר $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### צפיות
כעת נדרשות צפיות לחישוב ערכי הציפייה. במקרה שלנו אנו מסתכלים על בסיסים אורתוגונליים לכל קיוביט, ומאפשרים לסיבוב ה-$Y$ הפרמטרי של הקיוביט הראשון לסרוק את בסיס המדידה כמעט ברציפות ביחס לבסיס הקיוביט השני. לכן נבחר את הצפיות $ZZ$, $ZX$, $XZ$, ו-$XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## שלב 2: אופטימיזציה של הבעיה לביצוע על חומרה קוונטית

כדי לצמצם את זמן ביצוע המשימה הכולל, פרימיטיבים מגרסה V2 מקבלים רק מעגלים וצפיות התואמות להוראות ולקישוריות הנתמכות על ידי המערכת היעד (המכונה מעגלים וצפיות בארכיטקטורת קבוצת ההוראות — ISA).

### מעגל ISA

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### צפיות ISA

באופן דומה, עלינו לשנות את הצפיות כדי להתאים אותן ל-Backend לפני הרצת משימות עם [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). ניתן לבצע את ההמרה באמצעות שיטת `apply_layout` של אובייקט `SparsePauliOp`.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## שלב 3: ביצוע באמצעות פרימיטיבים של Qiskit
כדי לבצע את הניסוי כולו בקריאה אחת ל-[`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
נוכל ליצור פרימיטיב [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) לחישוב ערכי הציפייה שלנו. שיטת `EstimatorV2.run()` מקבלת אוסף של `primitive unified blocs (PUBs)`. כל PUB הוא אוסף בפורמט `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)